# Python实例练习：用户分类

分析目的：
* 计算退货率
* 利用RFM模型进行用户分类
* 对比分析不同用户群体的各项指标，并给出优化建议。

# 1.导入相关数据库

In [36]:
import numpy as np
import pandas as pd
import plotly as py
import plotly.graph_objs as go
import cufflinks
from plotly.offline import iplot,init_notebook_mode
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

# 2.读取数据并理解

In [2]:
data=pd.read_csv('online_retail_data.csv',encoding="ISO-8859-1",dtype={'CustomerID':str})

In [3]:
print(data.shape)
print(data.info())

(541909, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
InvoiceNo      541909 non-null object
StockCode      541909 non-null object
Description    540455 non-null object
Quantity       541909 non-null int64
InvoiceDate    541909 non-null object
UnitPrice      541909 non-null float64
CustomerID     406829 non-null object
Country        541909 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 33.1+ MB
None


# 3.数据清洗

## 3.1.缺失值处理

In [4]:
#统计缺失率
data.apply(lambda x:np.sum(x.isnull())/len(x),axis=0)

InvoiceNo      0.000000
StockCode      0.000000
Description    0.002683
Quantity       0.000000
InvoiceDate    0.000000
UnitPrice      0.000000
CustomerID     0.249267
Country        0.000000
dtype: float64

* Description缺失率为0.26%，CustomerID缺失率为24.93%。

In [5]:
#Description字段对于分析结果影响不大，不做处理
#CustomerID字段为客户编号，为保证用户分类基数不变，不直接删除，选择填充为Unknown
data['CustomerID']=data['CustomerID'].fillna('Unknown')

## 3.2.重复值处理

In [6]:
# 删除整行相同的重复数据
data.drop_duplicates(inplace=True)

## 3.3.异常值处理

In [7]:
data.describe(include='all')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
count,536641,536641,535187,536641.000000,536641,536641.000000,536641,536641
unique,25900,4070,4223,NaN,23260,NaN,4373,38
top,573585,85123A,WHITE HANGING HEART T-LIGHT HOLDER,NaN,10/31/2011 14:41,NaN,Unknown,United Kingdom
freq,1114,2301,2357,NaN,1114,NaN,135037,490300
mean,NaN,NaN,NaN,9.620029,NaN,4.632656,NaN,NaN
std,NaN,NaN,NaN,219.130156,NaN,97.233118,NaN,NaN
min,NaN,NaN,NaN,-80995.000000,NaN,-11062.060000,NaN,NaN
25%,NaN,NaN,NaN,1.000000,NaN,1.250000,NaN,NaN
50%,NaN,NaN,NaN,3.000000,NaN,2.080000,NaN,NaN
75%,NaN,NaN,NaN,10.000000,NaN,4.130000,NaN,NaN


* 商品数量Quantity和商品单价UnitPrice存在负值。

In [8]:
#商品数量异常查看并处理
df=data[data['Quantity']<=0]
#计算负值比例
print('商品数量异常数据比例:',f'{df.shape[0]/data.shape[0]:.2f}')
#检查是否都是退货订单并查看
print(len([c for c in df['InvoiceNo'] if 'C'not in c]))
df[~df['InvoiceNo'].str.contains('C')][:]

商品数量异常数据比例: 0.02
1336


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
2406,536589,21777,NaN,-10,12/1/2010 16:50,0.0,Unknown,United Kingdom
4347,536764,84952C,NaN,-38,12/2/2010 14:42,0.0,Unknown,United Kingdom
7188,536996,22712,NaN,-20,12/3/2010 15:30,0.0,Unknown,United Kingdom
7189,536997,22028,NaN,-20,12/3/2010 15:30,0.0,Unknown,United Kingdom
7190,536998,85067,NaN,-6,12/3/2010 15:30,0.0,Unknown,United Kingdom
...,...,...,...,...,...,...,...,...
535333,581210,23395,check,-26,12/7/2011 18:36,0.0,Unknown,United Kingdom
535335,581212,22578,lost,-1050,12/7/2011 18:38,0.0,Unknown,United Kingdom
535336,581213,22576,check,-30,12/7/2011 18:38,0.0,Unknown,United Kingdom
536908,581226,23090,missing,-338,12/8/2011 9:56,0.0,Unknown,United Kingdom


In [9]:
#对商品数量为负数进一步分析，发现非退货订单的单价均为0。
df[~df['InvoiceNo'].str.contains('C')]['UnitPrice'].unique()

array([0.])

* 从输出结果可以看出，数量为负数的订单不都是以‘C’开头的退货订单，数据存在异常情况。进一步分析发现，非退货订单的单价均为0，推测该部分为剩余活动赠品回收。实际业务中应该将该部分数据导出，并与相关部门确认数据异常原因。

In [10]:
#商品单价异常查看并处理
df2=data[data['UnitPrice']<=0]
print(df2['UnitPrice'].value_counts())
df2[df2['UnitPrice']<0]

 0.00        2510
-11062.06       2
Name: UnitPrice, dtype: int64


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
299983,A563186,B,Adjust bad debt,1,8/12/2011 14:51,-11062.06,Unknown,United Kingdom
299984,A563187,B,Adjust bad debt,1,8/12/2011 14:52,-11062.06,Unknown,United Kingdom


In [11]:
df2[df2['UnitPrice']==0]['Quantity'].describe()

count     2510.000000
mean       -53.529880
std        540.739276
min      -9600.000000
25%        -32.750000
50%         -2.000000
75%          3.000000
max      12540.000000
Name: Quantity, dtype: float64

* 从输出结果看，商品单价小于等于0的数据共有2512条，其中2条为负数，'Description'表示其为坏账调整；2510条为0，商品数量有正有负，推测为活动赠品出库和活动赠品回收，实际业务中应找相关部门明确。

## 3.4.增加新变量

In [12]:
#增加购买金额（Price）=数量*单价
data['Price']=data['Quantity']*data['UnitPrice']

In [13]:
#订单日期切割
data['InvoiceDate']=pd.to_datetime(data['InvoiceDate'],errors='coerce')
data['time']=data['InvoiceDate'].dt.time
data['month']=data['InvoiceDate'].dt.month
data['day']=data['InvoiceDate'].dt.day
data['year']=data['InvoiceDate'].dt.year
data['InvoiceDate']=data['InvoiceDate'].dt.date
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Price,time,month,day,year
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01,2.55,17850,United Kingdom,15.30,08:26:00,12,1,2010
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01,3.39,17850,United Kingdom,20.34,08:26:00,12,1,2010
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01,2.75,17850,United Kingdom,22.00,08:26:00,12,1,2010
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01,3.39,17850,United Kingdom,20.34,08:26:00,12,1,2010
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01,3.39,17850,United Kingdom,20.34,08:26:00,12,1,2010


# 4.数据分析

## 4.1.退货订单分析

* 指标一：退货率=退货合计金额/合计金额

In [14]:
#通过上述对商品数量和单价的异常值分析，为保证数据质量，我们以‘C’开头的退货订单计算退货金额。
df=data[(data['InvoiceNo'].str.contains('C')) & (data['Quantity']<=0)]

In [15]:
#t退货金额合计
Refund=pd.pivot_table(data=df,index=['month'],columns='year',values=['Price'],aggfunc={'Price':np.sum})
Refund.fillna(0,inplace=True)

In [16]:
#合计金额
df2=data[(~data['InvoiceNo'].str.contains('C')) & (data['Quantity']>0) & (data['UnitPrice']>0)]
Revenue=pd.pivot_table(data=df2,index=['month'],columns='year',values=['Price'],aggfunc={'Price':np.sum})
Revenue.fillna(0,inplace=True)

In [17]:
#退货率计算
rate=round(Refund/Revenue,3)*100
rate=np.abs(rate)
s=np.abs(round(sum(df['Price'])/sum(df2['Price']),3)*100)
s

8.4

In [37]:
#数据可视化
rate.iplot(
    kind='bar',
    xTitle='月份',
    yTitle='退货率',
    title='各个月份的退货率',
    hline=dict(y=s,width=3,color='red'))

* 从柱状图可以看出，2011年1月份和12月份的退货率均远高于平均退货率8.4%，特别是2011年12月，高达32.2%。具体原因应结合运营策略和动作进行分析，此外，可对比分析近几年的退货率，看是否存在显著的外力因素。

## 4.2.用户分类

* RFM模型是衡量客户价值和客户创利能力的重要工具和手段。该机械模型通过一个客户的近期购买行为、购买的总体频率以及花了多少钱三项指标来描述该客户的价值状况。
    * R：最近一次消费(Recency)
    * F：消费频率(Frequency)
    * M：消费金额(Monetary)
* 依据这三个指标，可以把客户分成：
    * 重要价值客户:最近消费时间近、消费频次和消费金额都很高，必须是VIP啊!
    * 重要保持客户:最近消费时间较远，但消费频次和金额都很高，说明这是个一段时间没来的忠诚客户，我们需要主动和他保持联系。
    * 重要发展客户(101):最近消费时间较近、消费金额高，但频次不高，忠诚度不高，很有潜力的用户，必须重点发展。
    * 重要挽留客户(001):最近消费时间较远、消费频次不高，但消费金额高的用户，可能是将要流失或者已经要流失的用户，应当基于挽留措施。

### 4.2.1.R的计算

In [21]:
#计算用户最近一次消费日期
df2['InvoiceDate']=pd.to_datetime(df2['InvoiceDate'],errors='coerce')
R_day=df2.groupby('CustomerID')['InvoiceDate'].max()
#以数据日期最大值为基准计算用户最近一次消费间隔的天数：
R=(df2['InvoiceDate'].max()-R_day).dt.days
R=pd.DataFrame(R)
print(R.describe())
R.iplot(kind='hist',
        bins=50,
        vline=[dict(x=i, color='red', dash='dash', width=3) for i in R.quantile([0.25,0.5,0.75,1])['InvoiceDate']],
        xTitle='最近一次消费',
        linecolor='black',
        histnorm='percent',
        yTitle='百分比(%)',
        title='用户最近一次消费间隔统计')

       InvoiceDate
count  4339.000000
mean     92.038258
std     100.010502
min       0.000000
25%      17.000000
50%      50.000000
75%     141.500000
max     373.000000


* 可以看出，最近一次消费间隔天数平均为92天，标准差为100，波动较大。
* 用户数量整体随间隔天数增加而减少，50%的用户消费间隔天数都在50天以内，用户结构健康。
* 25%的用户间隔天数超过4个月，最长超过一年。

### 4.2.2.F的计算

In [22]:
#计算用户的消费频率(排除重复的订单)
F=df2.groupby('CustomerID')['InvoiceNo'].nunique()
F=pd.DataFrame(F)
print(F.describe())
F.iplot(kind='hist',
        bins=100,
        xTitle='消费频率',
        linecolor='black',
        histnorm='percent',
        yTitle='百分比(%)',
        title='用户消费频率统计')

         InvoiceNo
count  4339.000000
mean      4.600138
std      22.943499
min       1.000000
25%       1.000000
50%       2.000000
75%       5.000000
max    1428.000000


In [23]:
F[F<40].iplot(kind='hist',
        bins=40,
        xTitle='消费频率',
        linecolor='black',
        histnorm='percent',
        yTitle='百分比(%)',
        title='用户消费频率统计')

* 用户50%的人消费低于2次，用户整体消费频次较低。受极大值1428次的影响，整体直方图不明显。
* 75%的用户消费频次低于40次，受极大值影响，消费频次平均值为4.6次。

### 4.2.3.M的计算

In [24]:
#设置显示为浮点型
np.set_printoptions(suppress=True)
pd.set_option('display.float_format',lambda x: '%.2f'% x) 
#计算客户的消费金额
M=df2.groupby('CustomerID')['Price'].sum()
M=pd.DataFrame(M)
print(M.describe())
M.iplot(kind='hist',
        bins=100,
        xTitle='消费金额',
        linecolor='black',
        histnorm='percent',
        yTitle='百分比(%)',
        title='消费金额')

           Price
count    4339.00
mean     2452.66
std     28086.06
min         3.75
25%       306.50
50%       668.58
75%      1660.89
max   1754901.91


* 和消费频率类似，由于极大值1754901.91的存在，直方图不明显。
* 50%的用户消费金额低于668元。

### 4.2.4.对用户进行分级

In [25]:
#设置bin(可自行设置)，这里F和M把极大值都摒弃了
R_bins=[0,30,90,180,360,720]
F_bins=[1,2,5,10,20,500]
M_bins=[0,500,2000,5000,10000,200000]

In [26]:
#数据离散化
R_score=pd.cut(R['InvoiceDate'],R_bins,labels=[5,4,3,2,1],right=False)
F_score=pd.cut(F['InvoiceNo'],F_bins,labels=[1,2,3,4,5],right=False)
M_score=pd.cut(M['Price'],M_bins,labels=[1,2,3,4,5],right=False)

In [27]:
#合并数据
RFM=pd.merge(left=R_score,right=F_score,left_index=True,right_index=True,how='inner')
RFM=pd.merge(left=RFM,right=M_score,left_index=True,right_index=True,how='inner')

In [28]:
RFM.rename(columns={'InvoiceDate':'R_score','InvoiceNo':'F_score','Price':'M_score'},inplace=True)
RFM.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4339 entries, 12346 to Unknown
Data columns (total 3 columns):
R_score    4339 non-null category
F_score    4338 non-null category
M_score    4336 non-null category
dtypes: category(3)
memory usage: 47.2+ KB


In [29]:
#将数据类型转换成浮点型
for i in ['R_score','F_score','M_score']:
    RFM[i]=RFM[i].astype('float')

In [30]:
RFM.describe()

,R_score,F_score,M_score
count,4339.00,4338.00,4336.00
mean,3.82,2.03,1.89
std,1.17,1.00,0.95
min,1.00,1.00,1.00
25%,3.00,1.00,1.00
50%,4.00,2.00,2.00
75%,5.00,3.00,2.00
max,5.00,5.00,5.00


* 以均值作为阈值，将得分分成‘高’和‘低’。

In [31]:
RFM['R']=np.where(RFM['R_score']>RFM['R_score'].mean(),'高','低')
RFM['F']=np.where(RFM['F_score']>RFM['F_score'].mean(),'高','低')
RFM['M']=np.where(RFM['M_score']>RFM['M_score'].mean(),'高','低')
RFM['value']=RFM['R']+RFM['F']+RFM['M']

In [32]:
#定义函数
def customer_value(x):
    if x=='高高高':
        return "重要价值客户"
    elif x=='高低高':
        return "重要发展客户"
    elif x=="低高高":
        return "重要保持客户"
    elif x=='低低高':
        return '重要挽留客户'
    elif x=='高高低':
        return "一般价值客户"
    elif x=='高低低':
        return "一般发展客户"
    elif x=='低高低':
        return "一般保持客户"
    else:
        return "一般挽留客户"

In [33]:
RFM['customer_value']=RFM['value'].apply(customer_value)
bar=RFM['customer_value'].value_counts()

In [34]:
#数据可视化
bar.iplot(kind='bar',
         title='用户等级情况',
         xTitle='用户等级',
         yTitle='用户数量',
         color='green')

In [35]:
trace=[go.Pie(labels=bar.index,values=bar.values,hole=0.2)]
layout=go.Layout(title='用户等级比例图')
fig=go.Figure(data=trace,layout=layout)
iplot(fig)

结论：
* 本文将用户分成8个类别，实际业务中，要根据业务类型做更针对性的划分。根据柱状图和饼图可知：
* 该公司用户数最多的是重要价值客户和重要发展客户，约占用户数的50%。对于重要发展客户，应通过信息曝光和促销活动增加其购买频次。
* 一般挽留客户和一般发展客户次之，占比约40%。一般发展客户应做进一步分析，区分新用户和重新活跃的老用户户，对新客户推送优惠券或折扣信息风措施来增加新用户粘性，对重新活跃老用户，了解用户消费需求，及时推送相应产品信息。